In [ ]:

# http://dbpedia.org/page/Autauga_County%2C_Alabama

In [ ]:

import sys

command_str = '{sys.executable} -m pip uninstall matplotlib'.format(sys=sys)
print(command_str)
#!{command_str}

In [1]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [2]:

%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
%run ../../load_magic/dataframes.py

import scraping_utils
wikipedia = scraping_utils.wikipedia

from IPython.display import clear_output
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import pearsonr
from tqdm import tqdm_notebook as tqdm
from urllib import request
from urllib.parse import unquote
import networkx as nx
import numpy as np
import re
import seaborn as sns
import wikidata

notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
counties_df = s.load_object('counties_df')
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])

%pprint
dir()

D:\Documents\Repositories\notebooks\covid19\ipynb\WikiData Exploration.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']
Pretty printing has been turned OFF


['Config', 'In', 'Out', 'RandomForestClassifier', 'SequenceMatcher', 'Storage', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_ih', '_ii', '_iii', '_oh', 'bs', 'check_4_doubles', 'check_for_typos', 'clear_output', 'conjunctify_list', 'copyfile', 'counties_df', 'csv', 'datetime', 'encoding', 'example_iterrows', 'exit', 'filepath_regex', 'get_classifier', 'get_column_descriptions', 'get_data_structs_dataframe', 'get_datastructure_prediction', 'get_dir_tree', 'get_git_lfs_track_commands', 'get_importances', 'get_input_sample', 'get_ipython', 'get_max_rsquared_adj', 'get_module_version', 'get_notebook_path', 'get_page_tables', 'get_specific_gitignore_files', 'get_struct_name', 'humanize_bytes', 'io', 'ipykernel', 'json', 'jupyter_config_dir', 'math', 'notebook_path', 'notebookapp', 'np', 'nx', 'os', 'pd', 'pearsonr', 'pickle', 'plt', 'preprocess_data', 'print_all_files_ending_start


----
# wptools Library

In [4]:

import wptools

title_str = 'Autauga County, Alabama'
page_obj = wptools.page(title_str)
page_obj.get_query()

en.wikipedia.org (query) Autauga County, Alabama
en.wikipedia.org (imageinfo) File:Autauga County Courthouse March...
Autauga County, Alabama (en) data
{
  WARNINGS: <dict(1)> extracts
  aliases: <list(1)> Autauga County, Alabama
  assessments: <dict(2)> United States, Alabama
  description: county in Alabama, United States
  extext: <str(228)> **Autauga County** is a county located in the...
  extract: <str(247)> <p><b>Autauga County</b> is a county located...
  image: <list(2)> {'kind': 'query-pageimage', 'file': 'File:Autau...
  label: Autauga County
  length: 27,167
  links: <list(277)> 1820 United States Census, 1830 United States...
  modified: <dict(1)> page
  pageid: 76923
  random: Time Today
  redirects: <list(7)> {'pageid': 85079, 'ns': 0, 'title': 'Autaug...
  requests: <list(2)> query, imageinfo
  title: Autauga County, Alabama
  url: https://en.wikipedia.org/wiki/Autauga_County,_Alabama
  url_raw: https://en.wikipedia.org/wiki/Autauga_County,_Alabama?a...
  wikibase: Q156

In [5]:

page = wptools.page(wikibase='Q156168')
page.get_wikidata()

www.wikidata.org (wikidata) Q156168
www.wikidata.org (labels) P6766|Q111266|Q1967620|Q499687|P31|P36|...
en.wikipedia.org (imageinfo) File:Autauga County Courthouse March...
Autauga County, Alabama (en) data
{
  aliases: <list(1)> Autauga County, Alabama
  claims: <dict(36)> P373, P17, P131, P138, P625, P882, P910, P242...
  description: county in Alabama, United States
  image: <list(1)> {'file': 'File:Autauga County Courthouse March ...
  label: Autauga County
  labels: <dict(50)> P6766, Q111266, Q1967620, Q499687, P31, P36, ...
  modified: <dict(1)> wikidata
  requests: <list(3)> wikidata, labels, imageinfo
  title: Autauga_County,_Alabama
  what: county of Alabama
  wikibase: Q156168
  wikidata: <dict(35)> Commons category (P373), country (P17), loc...
  wikidata_pageid: 157314
  wikidata_url: https://www.wikidata.org/wiki/Q156168
}



----
# wikidata Library

In [4]:

from wikidata.client import Client

print(['Client.{}'.format(fn) for fn in dir(Client) if not fn.startswith('_')])

['Client.cache_policy', 'Client.datavalue_decoder', 'Client.decode_datavalue', 'Client.entity_type_guess', 'Client.get', 'Client.guess_entity_type', 'Client.request']


In [4]:

from urllib.parse import quote
from wikidata.client import Client

title = 'Autauga County, Alabama'
url = './w/api.php?action=query&prop=imageinfo|info&inprop=url&iiprop=url|size|mime&format=json&titles={}'
url = url.format(quote(title))
client = Client()
result = client.request(url)

In [5]:

print(['result.{}'.format(fn) for fn in dir(result) if not fn.startswith('_')])

['result.clear', 'result.copy', 'result.fromkeys', 'result.get', 'result.items', 'result.keys', 'result.pop', 'result.popitem', 'result.setdefault', 'result.update', 'result.values']


In [8]:

for key, value in result.items():
    print(key, value)

batchcomplete 
query {'pages': {'-1': {'ns': 0, 'title': 'Autauga County, Alabama', 'missing': '', 'contentmodel': 'wikibase-item', 'pagelanguage': 'en', 'pagelanguagehtmlcode': 'en', 'pagelanguagedir': 'ltr', 'fullurl': 'https://www.wikidata.org/wiki/Autauga_County,_Alabama', 'editurl': 'https://www.wikidata.org/w/index.php?title=Autauga_County,_Alabama&action=edit', 'canonicalurl': 'https://www.wikidata.org/wiki/Autauga_County,_Alabama'}}}


In [5]:

url = './wiki/Special:EntityData/{}.json'.format(id)
result = Client.request(url)

Signature: Client.request(self, path: str) -> Union[bool, int, float, str, Mapping[str, Union[bool, int, float, str, Mapping[str, object], Sequence]], Sequence[Union[bool, int, float, str, Mapping[str, object], Sequence]]]
Docstring: <no docstring>
File:      c:\users\577342\appdata\local\continuum\anaconda3\lib\site-packages\wikidata\client.py
Type:      function



----

In [396]:

import pyperclip

pyperclip.copy(str(tuple((v, k) for k, v in idx_dict.items())))